In [1]:
# Import buit-in modules

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

%matplotlib notebook
plt.rcParams.update({'font.size': 17})

In [2]:
# Import my own modules
import sys
sys.path.insert(0, '../utils')
from mycvx import *
from myplots import *

## 6.1

In [3]:
Q1 = sp.MatrixSymbol('Q1', 4, 4)
Q2 = sp.MatrixSymbol('Q2', 4, 4)
Q3 = sp.MatrixSymbol('Q3', 4, 4)
Q4 = sp.MatrixSymbol('Q4', 4, 4)
Q = sp.BlockMatrix([[Q1, Q2, Q3, Q4],
                   [Q2, Q1, Q2, Q3],
                   [Q3, Q2, Q1, Q2],
                   [Q4, Q3, Q2, Q1]])

In [4]:
Q1_num = sp.Matrix([[12, 8, 7, 6],
                    [8, 12, 8, 7],
                    [7, 8, 12, 8],
                    [6, 7, 8, 12]])

Q2_num = sp.Matrix([[3, 2, 1, 0],
                    [2, 3, 2, 1],
                    [1, 2, 3, 2],
                    [0, 1, 2, 3]])

Q3_num = sp.Matrix([[2, 1, 0, 0],
                    [1, 2, 1, 0],
                    [0, 1, 2, 1],
                    [0, 0, 1, 2]])

Q4_num = sp.eye(4)

Q = Q.as_explicit()
Q = Q.evalf(subs={Qk: Qk_num for Qk, Qk_num in zip([Q1,Q2,Q3,Q4],[Q1_num,Q2_num,Q3_num,Q4_num])})

In [5]:
b = -sp.Matrix([1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0])

In [6]:
def loss1(x):
    return (x.T*Q*x)/2 + b.T*x

In [7]:
x0 = np.zeros((16,1))
xHist = {}
fxHist = {}
nbEvalList = {}

print('\nUsing \'Conjugate gradient\' - algo 6.2')
xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss1, conjugateMethod, \
                                                eps=1e-6)
print('\nx min', xHist[-1])
print('\nf(x min)', fxHist[-1])
print('\nnb evals', nbEvalList)


Using 'Conjugate gradient' - algo 6.2

x min [ 0.03423704  0.02423337  0.02423337  0.03423704 -0.00143237 -0.02088798
 -0.02088798 -0.00143237  0.03321913  0.02738606  0.02738606  0.03321913
 -0.00663057 -0.00486331 -0.00486331 -0.00663057]

f(x min) [[-0.1190756]]

nb evals {'nbFunEval': 9, 'nbHessEval': 9, 'nbGradEval': 9}


In [5]:
def loss2(x):
    return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

res = minimize(loss2, (0,0), method='nelder-mead', options={'xtol': 1e-6, 'disp': False})
loss1Min = res.x
print('xmin=',res.x)
print('f(xmin)=',res.fun)

xmin= [ 1.00000007  1.00000011]
f(xmin)= 3.43174783409e-14


In [4]:
x0s = np.array(((-2.0, 2.0), (2.0, -2.0), (-2.0, -2.0),))
xHist = {}
fxHist = {}
nbEvalList = {}
   
print('Using Fletcher-Reeves algorithm w/ line search w/ params alpha=0.01,beta=0.3')
for x0 in x0s:
    xHist = {}
    fxHist = {}
    nbEvalList = {}
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, conjugateMethod, eps=1e-6,
                                                     lineSearchMethod=backtrackingLineSearch,
                                                     alpha=0.01,beta=0.3)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

Using Fletcher-Reeves algorithm w/ line search w/ params alpha=0.01,beta=0.3

[-2.  2.]
x min [ 1.58530273  2.47020939]
f(x min) 0.527267478574
nb evals {'nbFunEval': 143, 'nbGradEval': 14}

[ 2. -2.]
x min [ 0.99988941  0.99977846]
f(x min) 1.22442208237e-08
nb evals {'nbFunEval': 303, 'nbGradEval': 42}

[-2. -2.]
x min [ 0.07990972 -0.05195758]
f(x min) 1.18695830996
nb evals {'nbFunEval': 128, 'nbGradEval': 14}
